In [1]:
from feast import FeatureStore
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from joblib import dump

In [2]:
# Getting our FeatureStore
store = FeatureStore(repo_path=".")

In [3]:
# Retrieving the saved dataset and converting it to a DataFrame
training_df = store.get_saved_dataset(name="churn_dataset").to_df()

C:\Users\damia\AppData\Roaming\Python\Python311\site-packages\feast\feature_store.py:1178: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [4]:
training_df

,NumOfProducts,USER_ID,Geography_Spain,EstimatedSalary,HasCrCard,Gender,Exited,Tenure,IsActiveMember,Geography_France,Balance,event_timestamp,Geography_Germany,CreditScore,Age
0,2,3158,True,28269.86,0,1,0,5,0,False,0.00,2023-01-01 00:00:00+00:00,False,574,34
1,2,2278,False,65731.53,1,0,0,6,0,True,0.00,2023-01-01 00:00:00+00:00,False,732,43
2,1,6187,False,178705.45,0,1,1,5,1,True,0.00,2023-01-01 00:00:00+00:00,False,618,37
3,2,7415,False,78650.95,1,0,0,2,1,False,99169.67,2023-01-01 00:00:00+00:00,True,737,45
4,1,8243,False,93332.64,0,0,1,2,1,True,0.00,2023-01-01 00:00:00+00:00,False,478,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,8470,False,197688.50,1,1,0,5,1,True,183135.39,2023-12-31 00:00:00+00:00,False,720,71
9996,2,6552,True,149195.44,0,0,0,5,1,False,0.00,2023-12-31 00:00:00+00:00,False,525,30
9997,1,2295,False,118782.06,1,1,0,9,0,True,131765.72,2023-12-31 00:00:00+00:00,False,591,33
9998,2,9796,False,105568.60,1,1,0,2,0,True,0.00,2023-12-31 00:00:00+00:00,False,506,43


In [5]:
# Separating the features and labels
labels = training_df['Exited']
features = training_df.drop(
    labels=['Exited', 'event_timestamp', "USER_ID"],
    axis=1)

In [6]:


# Dividi i dati
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(features, labels, test_size=0.2, random_state=42,stratify=labels)


In [7]:
# Splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    labels,
                                                    stratify=labels)

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Inizializza il modello
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Addestra il modello
model.fit(X=X_train_1[sorted(X_train_1)], y=y_train_1)

RandomForestClassifier(random_state=42)

In [9]:
# Creating and training LogisticRegression
reg = LogisticRegression()
reg.fit(X=X_train[sorted(X_train)], y=y_train)

C:\Users\damia\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [10]:
# Saving the model
dump(value=reg, filename="model_2.joblib")

['model_2.joblib']

In [11]:
dump(model, "random_forest_model.pkl")

['random_forest_model.pkl']